In [1]:
import graphene
import requests
import json
import pandas as pd

In [2]:
import graphene

class Query(graphene.ObjectType):
    hello = graphene.String(name=graphene.String(default_value="World"))

    def resolve_hello(self, info, name):
      return 'Hello ' + name

schema = graphene.Schema(query=Query)
result = schema.execute('{ hello }')
print(result.data['hello']) # "Hello World"

Hello World


# Functions

In [3]:


def make_vote_df_from_proposal_id(proposal_id):
    url = 'https://hub.snapshot.org/graphql/'
    vote_query = create_vote_query(proposal_id)
    vote_response = requests.post(url, json={'query': vote_query})
    print(proposal_response.status_code,proposal_response.reason,proposal_id)
    vote_json = vote_response.json()
    vote_df = vote_json_to_df(vote_json)
    return vote_df
    
def create_vote_query(proposal_id):
    vote_query = '''
    query Votes {
      votes (
        first: 9999999
        skip: 0
        where: {
          proposal: "'''+ proposal_id + '''"
        }
        orderBy: "created",
        orderDirection: desc
      ) {
        id
        ipfs
        voter
        created
        space {
          id
        }
        proposal {
          id
        }
        choice
        metadata
        vp
        vp_by_strategy
        vp_state
      }
    }
    '''
    return vote_query
def vote_json_to_df(vote_json):
    row_as_series_list=[]
    for vote in vote_json['data']['votes']:
        id = vote['id']
        ipfs = vote['ipfs']
        voter = vote['voter']
        created = vote['created']
        space = vote['space']['id']
        proposal = vote['proposal']['id']
        choice = vote['choice']
        #metadata = vote['metadata']: {},
        vp = vote['vp']
        vp_by_strategy = tuple(vote['vp_by_strategy'])
        vp_state = vote['vp_state']
        row_as_series  = pd.Series(
            data=(
                id,
                ipfs,
                voter,
                created,
                space,
                proposal,
                choice,
                #metadata,
                vp,
                vp_by_strategy,
                vp_state,
                ),
            index=(
                'vote_id',
                'ipfs',
                'voter',
                'created',
                'space',
                'proposal',
                'choice',
                #'metadata',
                'vp',
                'vp_by_strategy',
                'vp_state',
            )
        )
        row_as_series_list.append(row_as_series)
    return_df = pd.DataFrame(row_as_series_list)
    return return_df
    

In [4]:
proposal_query = '''
query Proposals {
  proposals (
    first: 10000000
    where: {
      space_in: ["index-coop.eth"],
      state: "closed"
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    scores
    scores_by_strategy
    scores_state
    scores_total
    scores_updated
    votes
    space {
      id
      name
    }
  }
}
'''
#print(proposal_query)

In [5]:
url = 'https://hub.snapshot.org/graphql/'
proposal_response = requests.post(url, json={'query': proposal_query})


In [6]:
print(proposal_response.status_code,proposal_response.reason)

200 OK


In [7]:
proposal_json = proposal_response.json()
print(len(proposal_json['data']['proposals']))
with open('all_proposals_20211120.json', 'w') as f:
    json.dump(proposal_json, f)

221


In [8]:
proposal_json['data']['proposals'][0]

{'id': '0x940fea8c7965da18b12fc6a081478204d41cda975ca8acdf220a17b0b99193aa',
 'title': 'IIP-105 DG1: Launch MATIC 2x Flexible Leverage Index (Polygon)',
 'body': "This is the first of two votes required for a new product to be onboarded to the Index Coop.\n\nA passing vote here signals the Index Coop's intent to further research the MATIC 2x Flexible Leverage Index (Polygon) feasibility resulting in a product prioritization score.\n\nPassing Thresholds:\n\n* 5% or more of circulating supply must participate (2,181,385*0.05 = 109 069)\n* 50% or more of participating tokens must vote FOR\n\nFOR - Pass MATIC 2x Flexible Leverage Index (Polygon) Index through Decision Gate 1. Begin work to research & assign MATIC 2x Flexible Leverage Index (Polygon) a product prioritization score.\n\nAGAINST - Do not pass MATIC 2x Flexible Leverage Index (Polygon) Index through Decision Gate 1.\n\nDG1 is a signaling step to gauge community sentiment. The full-text MATIC 2x Flexible Leverage Index (Polygon)

In [ ]:
votes_df_list = []
for proposal in proposal_json['data']['proposals']:
    print(proposal['state'],proposal['choices'])
    #print(proposal.keys())
    if proposal['state'] != 'closed':
        continue
    else:
        votes_for_proposal_df = make_vote_df_from_proposal_id(proposal['id'])
        votes_df_list.append(votes_for_proposal_df)
    

In [10]:
all_votes_df = pd.concat(votes_df_list,ignore_index=True)
all_votes_df['created'] = pd.to_datetime(all_votes_df['created'],unit='s')

ValueError: No objects to concatenate

In [82]:
all_votes_df['created'] = pd.to_datetime(all_votes_df['created'],unit='s')

In [85]:
all_votes_df.head()

,vote_id,ipfs,voter,created,space,proposal,choice,vp,vp_by_strategy,vp_state
0,0x4c35589e3b6c3d21ef953a45706e4521d281116ab7be...,QmPoR2Pb2FxNA85ymWmSR1B63SvkYxXsTqf6iWt5nRRPvr,0xaAa604E8f5318373D1785766D4f7310AF6CC4891,2021-11-18 17:10:05,index-coop.eth,0x940fea8c7965da18b12fc6a081478204d41cda975ca8...,1,22950.433092,"(0, 22950.433092108844)",final
1,0x610b571fa3bfc27980f68ecc18718a89a0c687157079...,QmcTZciba6W2tkGzXYho2ieECojtHxWpehmf8Qg8pxrr25,0x9829C7157889EE198cBc40208a8ABb29b3Ad2A2d,2021-11-18 16:54:25,index-coop.eth,0x940fea8c7965da18b12fc6a081478204d41cda975ca8...,1,200.000000,"(200, 0)",final
2,0xe716442936922d7c664e35b822b19e6fd95992d0cc60...,QmZ9TeP4GMHRtZNm2KaTjGJPNyi4BvLwG53Bc7WZftMrLh,0xBd31D988151DE348C928254D5622148Fdf7AF73C,2021-11-18 16:10:11,index-coop.eth,0x940fea8c7965da18b12fc6a081478204d41cda975ca8...,1,6313.927416,"(6313.9274159834395, 0)",final
3,0xfc7ea7360964d40f8d1b48a7fcc38bba2e8e92e93440...,QmNg7wwudkAwzbVhyySz5U2vAVGng3ASAB55axykSb5tcz,0x4F4538e2553C61BCE747f73C5fE8133D4B383Dda,2021-11-18 15:12:43,index-coop.eth,0x940fea8c7965da18b12fc6a081478204d41cda975ca8...,1,465.225939,"(465.2259387604784, 0)",final
4,0x3faf87b652187d4614fe28eaafa91532bea119941574...,QmPtXXoyyYrKPRRuDB9d4coRKw1r2Gm8vXgTufc6EEqoys,0x37B2691f9129a879913C5775307bEF89A77815D4,2021-11-18 14:57:18,index-coop.eth,0x940fea8c7965da18b12fc6a081478204d41cda975ca8...,1,693.016741,"(693.0167413625621, 0)",final


In [83]:
all_votes_df.to_json('all_votes_df.json')

# Scratchwork

In [22]:
proposal_id = '0x940fea8c7965da18b12fc6a081478204d41cda975ca8acdf220a17b0b99193aa'

In [38]:
vote_query = '''
query Votes {
  votes (
    first: 9999999
    skip: 0
    where: {
      proposal: "'''+ proposal_id + '''"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    ipfs
    voter
    created
    space {
      id
    }
    proposal {
      id
    }
    choice
    metadata
    vp
    vp_by_strategy
    vp_state
  }
}
'''

In [40]:
print(vote_query)


query Votes {
  votes (
    first: 9999999
    skip: 0
    where: {
      proposal: "0x940fea8c7965da18b12fc6a081478204d41cda975ca8acdf220a17b0b99193aa"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    ipfs
    voter
    created
    space {
      id
    }
    proposal {
      id
    }
    choice
    metadata
    vp
    vp_by_strategy
    vp_state
  }
}



In [82]:
url = 'https://hub.snapshot.org/graphql/'
r = requests.post(url, json={'query': vote_query})
print(r.status_code,r.reason)

200 OK


In [83]:
vote_json = r.json()

In [84]:
vote_json['data']['votes']

[{'id': '0x142418f28d21b3ba356caa91025d45ee84558dde1e7af7b9042e00f27bf83048',
  'ipfs': 'QmYpBuRKXYdLfgaCMKnzuwqEBmj9EijWTpNL5hnmzr6oxG',
  'voter': '0x53C40473DcdFd927C4201cCFE24E314a7D7C3584',
  'created': 1635623597,
  'space': {'id': 'index-coop.eth'},
  'proposal': {'id': 'QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa'},
  'choice': 1,
  'metadata': {},
  'vp': 88.85845661274872,
  'vp_by_strategy': [88.85845661274872, 0],
  'vp_state': 'final'},
 {'id': '0x7b696f7df1ec720a1b976d06ffed895e8b7ed0cb8872392f54ad221b3e10eb92',
  'ipfs': 'QmUrs4tt8a49ZQPasxXvrQ2xGGWb9ZhSUTBU7rqMEE8KZj',
  'voter': '0x0100e4f2a841e773564c333f889ab6D6Bd5FcB22',
  'created': 1635602262,
  'space': {'id': 'index-coop.eth'},
  'proposal': {'id': 'QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa'},
  'choice': 1,
  'metadata': {},
  'vp': 8912.113703479938,
  'vp_by_strategy': [753.0289283021797, 8159.084775177758],
  'vp_state': 'final'},
 {'id': '0x1ead0f3d9ebd31a303db747fd12b89b7c701ec4d2d56b5eddcd82ae39ab

In [43]:
def vote_json_to_df(vote_json):
    row_as_series_list=[]
    for vote in vote_json['data']['votes']:
        id = vote['id']
        ipfs = vote['ipfs']
        voter = vote['voter']
        created = vote['created']
        space = vote['space']['id']
        proposal = vote['proposal']['id']
        choice = vote['choice']
        #metadata = vote['metadata']: {},
        vp = vote['vp']
        vp_by_strategy = tuple(vote['vp_by_strategy'])
        vp_state = vote['vp_state']
        row_as_series  = pd.Series(
            data=(
                id,
                ipfs,
                voter,
                created,
                space,
                proposal,
                choice,
                #metadata,
                vp,
                vp_by_strategy,
                vp_state,
                ),
            index=(
                'id',
                'ipfs',
                'voter',
                'created',
                'space',
                'proposal',
                'choice',
                #'metadata',
                'vp',
                'vp_by_strategy',
                'vp_state',
            )
        )
        row_as_series_list.append(row_as_series)
    return_df = pd.DataFrame(row_as_series_list)
    return return_df
    

In [79]:
votes_df = vote_json_to_df(vote_json)

In [80]:
votes_df

,id,ipfs,voter,created,space,proposal,choice,vp,vp_by_strategy,vp_state
0,0x142418f28d21b3ba356caa91025d45ee84558dde1e7a...,QmYpBuRKXYdLfgaCMKnzuwqEBmj9EijWTpNL5hnmzr6oxG,0x53C40473DcdFd927C4201cCFE24E314a7D7C3584,1635623597,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,88.858457,"(88.85845661274872, 0)",final
1,0x7b696f7df1ec720a1b976d06ffed895e8b7ed0cb8872...,QmUrs4tt8a49ZQPasxXvrQ2xGGWb9ZhSUTBU7rqMEE8KZj,0x0100e4f2a841e773564c333f889ab6D6Bd5FcB22,1635602262,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,8912.113703,"(753.0289283021797, 8159.084775177758)",final
2,0x1ead0f3d9ebd31a303db747fd12b89b7c701ec4d2d56...,QmdhDd2vLrG2kCrFutiRRbyNDz1zsz5xEvrt7WGv7qCg5s,0x6E07663b21bD087a53739BC2109aC8360ebB0270,1635584931,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,2,77.069316,"(77.06931592224655, 0)",final
3,0xd100c55917933d152f013829aa9c74dd7fc58ad8dfec...,QmZ2m2HS9rJ4ifkVdPuTaDHvtdaU33Qn5v4G3bZw3wXa7j,0x7f5Fe0Cf45aD50b4E80Fe66a87f692B7c006bc3e,1635574698,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,1.009999,"(1.009998885338774, 0)",final
4,0x2de599e9ea9c9217d7094fc48f6044dba3c7bc285582...,QmRUHT2ATZt7iEVgRpV99HzApKanCMXZsMHuaaMt8cqBv8,0x83C9440dc34DA00c47A0d4dC2b598d7BDB1b53F7,1635553233,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,5.694520,"(5.69451958746213, 0)",final
5,0xace7b8e7c6432d2a14425c1350329708a1551163463f...,QmeC4qVMseA4avgGfTeLBr2zh557ahGG7cKQj8TTKms89E,0x4AdA1B9D9fe28aBd9585f58cfEeD2169A39e1c6b,1635549257,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,148.161771,"(148.16177066678787, 0)",final
6,0x937907e6e9dc2dbb3956a5af44c8c526f98045f9c6aa...,QmcdZpUjhB38x1T6FeBfgZLjLKwTZQrGTedXzuffbtZPcy,0x070341aA5Ed571f0FB2c4a5641409B1A46b4961b,1635529404,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,8793.329084,"(0, 8793.329084453937)",final
7,0x0e8cce791593cd24705fa42bad14e1c934d17a14da9b...,QmfJaGHnmLdv2CTdTTk8YWy1uTnZTLrYCGxNpoS4KRRVGv,0x4f49D938c3Ad2437c52eb314F9bD7Bdb7FA58Da9,1635521733,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,104.830000,"(104.83, 0)",final
8,QmbyBUw4mfLyaamtpAzA8DitX5HxRuo6B2ikfWQFDoowL9,QmbyBUw4mfLyaamtpAzA8DitX5HxRuo6B2ikfWQFDoowL9,0x981CaC16246D9641bE07BbA6E19E304b74B173BC,1635490934,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,633.936943,"(40, 593.9369434653063)",final
9,0x8b1842fc9d30f1b7dcf3a5a5f2836af4d393e1e0d6c4...,Qmc4n4HsWExGEe6jLLKVHTu4TgiCsWuwPYdKQLdAKYLM4a,0x6a1a3eCd04DF0fF2d7cb25584052CDD173fC609B,1635461353,index-coop.eth,QmbbpCkypmXYZbBf65N9syoAWsEHXZPo4QgsHapZDC6hZa,1,37.270088,"(37.270087876477305, 0)",final


In [48]:
vote = vote_json['data']['votes'][0]